In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import math
import sklearn
from scipy import stats
import statsmodels.formula.api as sm

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn import linear_model
import patsy
import statsmodels.formula.api as smf
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from __future__ import division
%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline

# Q1 Baseline Model 
## pt a Regression

In [690]:
data = pd.read_excel("NRDF.xlsx")
SaleOver2k = (np.array([data['SalePrice']>200000])*1).reshape(len(data),1)
data = data.assign(SaleOver2k = SaleOver2k)

In [691]:
train, test = train_test_split(data, test_size=0.2)
test.to_csv('test.csv')
data = train

In [692]:
def dataCVforLinearModels(data,target,log):
    training, testing = train_test_split(data, test_size=0.2)
    if log == 'log':
        training = training.set_index([list(range(len(training)))])
        for i in range(len(training["SalePrice"])):
            training.loc[i,('SalePrice')] = math.log(training.loc[i,('SalePrice')])
        testing = testing.set_index([list(range(len(testing)))])
        for i in range(len(testing["SalePrice"])):
            testing.loc[i,('SalePrice')] = math.log(testing.loc[i,('SalePrice')])
        test_x=testing.drop(['SalePrice'],axis = 1)
        test_y=testing['SalePrice']
    X = training.drop([target], axis=1)
    Y = training[target]
    test_x=testing.drop([target], axis=1)
    test_y=testing[target]
    return training,testing,X,Y,test_x,test_y

def linearRegression(data,target):
    lm = smf.ols(target+' ~ '+' + '.join(list(data.drop([target], axis=1))), data=data).fit()
    return lm

def logisticRegression(data,target):
    lr = LogisticRegression()
    lr.fit(train.drop([target],axis=1),train[target])
    return lr

def CVforRegModels(data,target,lm,log):
    insample_r2=[]
    outsample_r2=[]
    outsample_rsme=[]
    for i in range (5):
        training,testing,X,Y,test_x,test_y = dataCVforLinearModels(data,target,log)
        if lm == "linear":
            lm = linearRegression(training,target)
        elif lm == "logistic": 
            lm = logisticRegression(training,target)
        rsq = 1. - np.sum((lm.predict(X)-Y)**2)/np.sum((
        Y.mean()-Y)**2)
        insample_r2.append(rsq)
        rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
        test_y.mean()-test_y)**2)
        outsample_r2.append(rsq)
        rsme = sqrt(mean_squared_error(test_y, lm.predict(test_x)))
        outsample_rsme.append(rsme)
    print "Estimated out of sample Root mean squared error for this model is ", sum(outsample_rsme)/5
    print 'Estimated out of sample R-square for this model is',sum(outsample_r2)/5
    print 'Estimated in sample R-square for this model is',sum(insample_r2)/5
    
def CVforClassificationModels(data,target,lm,log):
    insample_r2=[]
    outsample_r2=[]
    outsample_rsme=[]
    for i in range (5):
        training,testing,X,Y,test_x,test_y = dataCVforLinearModels(data,target,log)
        if lm == "linear":
            lm = linearRegression(training,target)
        elif lm == "logistic": 
            lm = logisticRegression(training,target)
        ab = sum((lm.predict(X)>0.5)*1*Y )/sum(Y)
        insample_r2.append(ab)
        ac = sum((lm.predict(test_x)>0.5)*1*test_y )/sum(test_y)
        outsample_r2.append(ac)
    print 'Estimated out of sample accuracy for this model is',sum(outsample_r2)/5
    print 'Estimated in sample accuracy for this model is',sum(insample_r2)/5
    
def FinalClassificationTest(data,target,lm,log):
    insample_r2=[]
    outsample_r2=[]
    outsample_rsme=[]
    for i in range (5):
        training,testing,X,Y,test_x,test_y = dataCVforLinearModels(data,target,log)
        if lm == "linear":
            lm = linearRegression(training,target)
        elif lm == "logistic": 
            lm = logisticRegression(training,target)
        ab = sum((lm.predict(X)>0.5)*1*Y )/sum(Y)
        insample_r2.append(ab)
        ac = sum((lm.predict(test_x)>0.5)*1*test_y )/sum(test_y)
        outsample_r2.append(ac)
    print 'Final Test Result for accurancy is ',sum(outsample_r2)/5

In [693]:
CVforRegModels(train.drop(['SaleOver2k'], axis=1),'SalePrice','logistic','none')

KeyboardInterrupt: 

## pt a Classfication

In [ ]:
lm = linearRegression(data.drop(['SalePrice'], axis=1),'SaleOver2k')
CVforClassificationModels(data,'SaleOver2k',lm,'none')

## pt b 

##### 1-0 Loss function is not a useful point of reference in this case for example: it might not punish the price
##### large enough and a sale price way below 200,000 and one that is close to 200,000 are totally different, but 1-0
##### loss function rates them the same way

## pt c Regression

In [ ]:
data2 = pd.read_excel("NewData.xlsx")
singlefeatures = data2.columns[:-1]
features = list(singlefeatures)
for i in range(len(features)):
    features[i] = "".join(features[i].split())
    features[i] = features[i].replace('.','DOT')
singlefeatures = features
features.append('SalePrice')
data2.columns = features
train2, test2 = train_test_split(data2, test_size=0.2,random_state=3)

In [ ]:
#results from the selected variable model
insample_r2=[]
outsample_r2=[]
outsample_rsme=[]
for i in range (5):
    training, testing = train_test_split(train2, test_size=0.2)
    X = training.drop(['SalePrice'],axis = 1)
    Y = training['SalePrice']
    test_x=testing.drop(['SalePrice'],axis = 1)
    test_y=testing['SalePrice']
    lm = smf.ols('SalePrice ~ '+' + '.join(list(X)), data=training).fit()
    insample_r2.append(lm.rsquared)
    rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((test_y.mean()-test_y)**2)
    outsample_r2.append(rsq)
    rsme = sqrt(mean_squared_error(test_y, lm.predict(test_x)))
    outsample_rsme.append(rsme)
print "Estimated out of sample Root mean squared error for this model is ", sum(outsample_rsme)/5
print 'Estimated out of sample R-square for this model is',sum(outsample_r2)/5
print 'Estimated in sample R-square for this model is',sum(insample_r2)/5

Baseline model performs slightly better.

## pt c classification

In [ ]:
X = data.drop(['SalePrice','SaleOver2k'], axis=1)
Y = data['SaleOver2k']
alpha=np.arange(20,40,1)
singlefeatures = list(X)
lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
Importantfeatures = np.array(singlefeatures)[np.abs(lasso.coef_)>1e-10]

In [ ]:
lr = LogisticRegression()
lr.fit(data[Importantfeatures],data['SaleOver2k'])
Importantfeatures = list(set(Importantfeatures))
Importantfeatures.append('SaleOver2k')
CVforClassificationModels(data[Importantfeatures],'SaleOver2k',lr,"none")

#### they do not having a great in-sample score and 
#### out-sample is almost random. It does not perform as
#### good as baseline model (almost random)

# Q2 Baseline Model 
## pt a Best Regression Model

In [ ]:
#Linear regression with Lasso
insample_r2=[]
outsample_r2=[]
outsample_rsme=[]
for i in range (5):
    training, testing = train_test_split(train, test_size=0.2)
    X = training.drop(['SalePrice','SaleOver2k'], axis=1)
    Y = training['SalePrice']
    test_x=testing.drop(['SalePrice','SaleOver2k'], axis=1)
    test_y=testing['SalePrice']
    alpha=np.arange(0,30,0.1)
    singlefeatures = list(X)
    lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
    lassofeatures = np.array(singlefeatures)[np.abs(lasso.coef_)>1e-10]
    lm = smf.ols('SalePrice ~ '+' + '.join(lassofeatures), data=training).fit()
    insample_r2.append(lm.rsquared)
    rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
    outsample_r2.append(rsq)
    rsme = sqrt(mean_squared_error(test_y, lm.predict(test_x)))
    outsample_rsme.append(rsme)
print "Estimated out of sample Root mean squared error for this model is ", sum(outsample_rsme)/5
print 'Estimated out of sample R-square for this model is',sum(outsample_r2)/5
print 'Estimated in sample R-square for this model is',sum(insample_r2)/5

In [ ]:
#Linear regression with Ridge
insample_r2=[]
outsample_r2=[]
outsample_rsme=[]
for i in range (5):
    training, testing = train_test_split(train, test_size=0.2)
    X = training.drop(['SalePrice','SaleOver2k'], axis=1)
    Y = training['SalePrice']
    test_x=testing.drop(['SalePrice','SaleOver2k'], axis=1)
    test_y=testing['SalePrice']
    alpha=np.arange(0,30,0.1)
    ridge = linear_model.RidgeCV(alphas=alpha,cv =5).fit(X,Y)
    R2=ridge.score(X,Y)
    insample_r2.append(R2)
    Prediction = ridge.predict(test_x)
    R2_out = 1. - np.sum((Prediction-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
    outsample_r2.append(R2_out)
    rsme = sqrt(mean_squared_error(test_y, Prediction))
    outsample_rsme.append(rsme)
print "Estimated out of sample Root mean squared error for this model is ", sum(outsample_rsme)/5
print 'Estimated out of sample R-square for this model is',sum(outsample_r2)/5
print 'Estimated in sample R-square for this model is',sum(insample_r2)/5

In [ ]:
#Linear regression with Lasso and log transformation
insample_r2=[]
outsample_r2=[]
outsample_rsme=[]
for i in range (5):
    training, testing = train_test_split(train, test_size=0.2)
    training = training.set_index([list(range(len(training)))])
    for i in range(len(training["SalePrice"])):
        training.loc[i,('SalePrice')] = math.log(training.loc[i,('SalePrice')])    
    X = training.drop(['SalePrice'],axis = 1)
    Y = training['SalePrice']
    testing = testing.set_index([list(range(len(testing)))])
    for i in range(len(testing["SalePrice"])):
        testing.loc[i,('SalePrice')] = math.log(testing.loc[i,('SalePrice')])
    test_x=testing.drop(['SalePrice'],axis = 1)
    test_y=testing['SalePrice']
    alpha=np.arange(0,30,0.1)
    singlefeatures = list(X)
    lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
    lassofeatures = np.array(singlefeatures)[np.abs(lasso.coef_)>1e-8]
    lm = smf.ols('SalePrice ~ '+' + '.join(lassofeatures), data=training).fit()
    insample_r2.append(lm.rsquared)
    rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
    outsample_r2.append(rsq)
    for i in range(len(testing["SalePrice"])):
        testing.loc[i,('SalePrice')] = math.exp(testing.loc[i,('SalePrice')])
    prediction = lm.predict(test_x)
    for i in range(len(prediction)):
        prediction[i]=math.exp(prediction[i])
    rsme = sqrt(mean_squared_error(test_y, prediction))
    outsample_rsme.append(rsme)
print "Estimated out of sample Root mean squared error for this model is ", sum(outsample_rsme)/5
print 'Estimated out of sample R-square for this model is',sum(outsample_r2)/5
print 'Estimated in sample R-square for this model is',sum(insample_r2)/5

## pt a Best Classfication Model

In [ ]:
#baseline
lm = linearRegression(train.drop(['SalePrice'], axis=1),'SaleOver2k')
CVforClassificationModels(train,'SaleOver2k',lm,'none')

In [ ]:
#knn
from sklearn.neighbors import KNeighborsClassifier 
train, test = train_test_split(data, test_size=0.2,random_state=3)
neigh = KNeighborsClassifier(n_neighbors=1) # 1 is the best one using CV
neigh.fit(train.drop(['SalePrice','SaleOver2k'],axis=1), train['SaleOver2k']) 
a = sum(neigh.predict(test.drop(['SalePrice','SaleOver2k'],axis=1))*test['SaleOver2k'])/sum(train['SaleOver2k'])
print 'Estimated out of sample accuracy for this model is ' + str(a)

In [ ]:
#selected variable model
data2 = pd.read_excel("NewData.xlsx")
SaleOver2k = (np.array([data2['SalePrice']>200000])*1).reshape(len(data2),1)
data2= data2.assign(SaleOver2k = SaleOver2k)
singlefeatures = data2.columns[:-2]
features = list(singlefeatures)
for i in range(len(features)):
    features[i] = "".join(features[i].split())
    features[i] = features[i].replace('.','DOT')
singlefeatures = features
features.append('SalePrice')
features.append('SaleOver2k')
data2.columns = features
train2, test2 = train_test_split(data2, test_size=0.2,random_state=3)

In [ ]:
#selected variable
lm = linearRegression(train2.drop(['SalePrice'], axis=1),'SaleOver2k')
CVforClassificationModels(train2,'SaleOver2k',lm,'none')

In [ ]:
#selected variable with log transformation
lm = linearRegression(train2.drop(['SalePrice'], axis=1),'SaleOver2k')
CVforClassificationModels(train2,'SaleOver2k',lm,'log')

In [ ]:
#all variables with logistic model
lr = LogisticRegression()
lr.fit(train.drop(['SalePrice','SaleOver2k'],axis=1),train['SaleOver2k'])
CVforClassificationModels(train.drop(['SalePrice'],axis=1),'SaleOver2k',lr,"none")

In [ ]:
data = data2

In [ ]:
data['ExterQual'] = data['ExterQual'].replace('Ex', 5)
data['ExterQual'] = data['ExterQual'].replace('Gd', 4)
data['ExterQual'] = data['ExterQual'].replace('TA', 3)
data['ExterQual'] = data['ExterQual'].replace('Fa', 2)
data['ExterQual'] = data['ExterQual'].replace('Po', 1)
data['ExterCond'] = data['ExterCond'].replace('Ex', 5)
data['ExterCond'] = data['ExterCond'].replace('Gd', 4)
data['ExterCond'] = data['ExterCond'].replace('TA', 3)
data['ExterCond'] = data['ExterCond'].replace('Fa', 2)
data['ExterCond'] = data['ExterCond'].replace('Po', 1)
data['BsmtQual'] = data['BsmtQual'].replace('Ex', 5)
data['BsmtQual'] = data['BsmtQual'].replace('Gd', 4)
data['BsmtQual'] = data['BsmtQual'].replace('TA', 3)
data['BsmtQual'] = data['BsmtQual'].replace('Fa', 2)
data['BsmtQual'] = data['BsmtQual'].replace('Po', 1)
data['BsmtQual'] = data['BsmtQual'].replace('No Basement', 0)
data['BsmtCond'] = data['BsmtCond'].replace('Ex', 5)
data['BsmtCond'] = data['BsmtCond'].replace('Gd', 4)
data['BsmtCond'] = data['BsmtCond'].replace('TA', 3)
data['BsmtCond'] = data['BsmtCond'].replace('Fa', 2)
data['BsmtCond'] = data['BsmtCond'].replace('Po', 1)
data['BsmtCond'] = data['BsmtCond'].replace('No Basement', 0)

In [ ]:
#all variables with logistic model with selected variables
lr = LogisticRegression()
lr.fit(data.drop(['SalePrice','SaleOver2k'],axis=1),data['SaleOver2k'])
CVforClassificationModels(data.drop(['SalePrice'],axis=1),'SaleOver2k',lr,"none")

# Q3 Prediction

In [ ]:
test = pd.read_csv("test.csv")

In [658]:
#out of sample prediction
lr = LogisticRegression()
lr.fit(data.drop(['SalePrice','SaleOver2k'],axis=1),data['SaleOver2k'])
FinalClassificationTest(data.drop(['SalePrice'],axis=1),'SaleOver2k',lr,"none")

Final Test Result for accurancy is  0.9314536961230777


In [669]:
#out of sample prediction
lr = LogisticRegression()
lr.fit(test.drop(['SalePrice','SaleOver2k'],axis=1),test['SaleOver2k'])
FinalClassificationTest(data.drop(['SalePrice'],axis=1),'SaleOver2k',lr,"none")

Final Test Result for accurancy is  0.9156626506024097


# Log Transformation


In [188]:
train = train.set_index([list(range(len(train)))])
for i in range(len(train["SalePrice"])):
    train.loc[i,('SalePrice')] = math.log(train.loc[i,('SalePrice')])

In [189]:
X = train.drop(['SalePrice'], axis=1)
Y = train['SalePrice']
alpha=np.arange(0,30,0.1)
singlefeatures = list(X)
lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
lassofeatures = np.array(singlefeatures)[np.abs(lasso.coef_)>1e-10]

In [190]:
test = test.set_index([list(range(len(test)))])
for i in range(len(test["SalePrice"])):
    test.loc[i,('SalePrice')] = math.log(test.loc[i,('SalePrice')])

In [193]:
lm = smf.ols('SalePrice ~ '+' + '.join(lassofeatures), data=train).fit()
print 'Here is in-sample R-square:'+str(lm.rsquared)
test_x=test.drop(['SalePrice'], axis=1)
test_y=test['SalePrice']
rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
print 'Here is the out-of-sample R-square:'+str(rsq)

Here is in-sample R-square:0.9212655271253929
Here is the out-of-sample R-square:0.87109017926


In [20]:
for i in range(len(test["SalePrice"])):
    test.loc[i,('SalePrice')] = math.exp(test.loc[i,('SalePrice')])

In [21]:
prediction = lm.predict(test_x)
for i in range(len(prediction)):
      prediction[i]=math.exp(prediction[i])

In [22]:
rms_test = sqrt(mean_squared_error(test['SalePrice'], prediction))
print "Root mean squared error of the prediction is ", rms_test

Root mean squared error of the prediction is  44478.1758102


# BASELINE MODEL

In [23]:
singlefeatures=train.drop(['SalePrice'], axis=1)

In [24]:
for i in range(len(train["SalePrice"])):
    train.loc[i,('SalePrice')] = math.exp(train.loc[i,('SalePrice')])

In [25]:
lm = smf.ols('SalePrice ~ '+' + '.join(singlefeatures), data=train).fit()
print 'Here is in-sample R-square:'+str(lm.rsquared)
rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
print 'Here is the out-of-sample R-square:'+str(rsq)

Here is in-sample R-square:0.8972564589848324
Here is the out-of-sample R-square:0.846172299948


In [26]:
rms_test = sqrt(mean_squared_error(test['SalePrice'], lm.predict(test_x)))
print "Root mean squared error of the prediction is ", rms_test

Root mean squared error of the prediction is  32407.6200487
